<a href="https://colab.research.google.com/github/isabel-sha/ml-ai/blob/main/Anonymize_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
from google.colab import files
import os
import pandas as pd
import random
import uuid
from datetime import timedelta
import string
import numpy as np
!pip install pandas openpyxl

In [2]:
# Upload file
uploaded = files.upload()

In [3]:
# Load the dataset
df = pd.read_excel('incident_metric.xlsx')

In [4]:
# Generate and anonymize incident numbers in the format INCxxxxxx
def generate_incident_number():
    return "INC" + str(random.randint(100000, 999999))
incident_number_map = {}
def anonymize_incident_number(original_number):
    if original_number not in incident_number_map:
        random_incident = generate_incident_number()
        while random_incident in incident_number_map.values():
            random_incident = generate_incident_number()
        incident_number_map[original_number] = random_incident
    return incident_number_map[original_number]
df['Number'] = df['Number'].apply(anonymize_incident_number)

In [5]:
# Anonymize requested for, assigned to, and value fields
def generate_random_string(length=6):
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))
assigned_to_map = {}
def anonymize_assigned_to_value_and_requested_for(row):
    assigned_to = row['Assigned to']
    value = row['Value']
    change = row['Change']
    requested_for = row['Requested For']
    if isinstance(requested_for, str) and requested_for.strip().lower() == "guest":
        requested_for = np.nan
    if pd.isna(assigned_to):
        randomized_assigned_to = assigned_to
    else:
        if assigned_to not in assigned_to_map:
            random_string = generate_random_string(6)
            assigned_to_map[assigned_to] = f"{random_string}"
        randomized_assigned_to = assigned_to_map[assigned_to]
    if change == 'Assigned to':
        if pd.isna(value):
            return randomized_assigned_to, value, requested_for
        if value == assigned_to:
            value = randomized_assigned_to
        elif value in assigned_to_map:
            value = assigned_to_map[value]
        else:
            random_string = generate_random_string(6)
            assigned_to_map[value] = f"{random_string}"
            value = assigned_to_map[value]
    if pd.isna(requested_for):
        randomized_requested_for = requested_for
    else:
        if requested_for == assigned_to:
            randomized_requested_for = randomized_assigned_to
        elif requested_for in assigned_to_map:
            randomized_requested_for = assigned_to_map[requested_for]
        else:
            random_string = generate_random_string(6)
            assigned_to_map[requested_for] = f"{random_string}"
            randomized_requested_for = assigned_to_map[requested_for]
    return randomized_assigned_to, value, randomized_requested_for
df[['Assigned to', 'Value', 'Requested For']] = df.apply(anonymize_assigned_to_value_and_requested_for, axis=1, result_type='expand')

In [6]:
# Anonymize business unit by assigning a random department name
def generate_random_string(length=3):
    return ''.join(random.choices(string.ascii_uppercase, k=length))
business_unit_map = {}
def anonymize_business_unit(original_unit):
    if pd.isna(original_unit):
        return np.nan
    if original_unit not in business_unit_map:
        random_string = generate_random_string(3)
        business_unit_map[original_unit] = f"Department {random_string}"
    return business_unit_map[original_unit]
df['Business unit'] = df['Business unit'].apply(anonymize_business_unit)

In [7]:
# Anonymize assignment group and value by assigning a random team name
assignment_group_map = {}
def anonymize_assignment_group_and_value(row):
    original_group = row['Assignment group']
    value = row['Value']
    change = row['Change']
    if pd.isna(original_group):
        randomized_group = original_group
    else:
        if original_group not in assignment_group_map:
            random_string = generate_random_string(3)
            assignment_group_map[original_group] = f"Team {random_string}"
        randomized_group = assignment_group_map[original_group]
    if change == 'Assignment Group':
        if pd.isna(value):
            return randomized_group, value
        if value == original_group:
            value = randomized_group
        elif value in assignment_group_map:
            value = assignment_group_map[value]
        else:
            random_string = generate_random_string(3)
            assignment_group_map[value] = f"Team {random_string}"
            value = assignment_group_map[value]
    return randomized_group, value
df[['Assignment group', 'Value']] = df.apply(anonymize_assignment_group_and_value, axis=1, result_type='expand')

In [8]:
# Randomly shift date/time columns
def shift_row_datetime(row, date_columns, max_days=7, max_hours=72, max_minutes=1440, max_seconds=86400):
    shift_days = random.randint(-max_days, max_days)
    shift_hours = random.randint(-max_hours, max_hours)
    shift_minutes = random.randint(-max_minutes, max_minutes)
    shift_seconds = random.randint(-max_seconds, max_seconds)
    for col in date_columns:
        if pd.notna(row[col]):
            row[col] = row[col] + pd.Timedelta(days=shift_days, hours=shift_hours, minutes=shift_minutes, seconds=shift_seconds)
    return row
def shift_dates(df, date_columns, max_days=7, max_hours=72, max_minutes=1440, max_seconds=86400):
    return df.apply(lambda row: shift_row_datetime(row, date_columns, max_days, max_hours, max_minutes, max_seconds), axis=1)
date_columns = ['Opened', 'Created', 'Updated', 'Resolved', 'Closed', 'Start', 'End']
df[date_columns] = df[date_columns].apply(pd.to_datetime, errors='coerce')
df = shift_dates(df, date_columns)

In [9]:
# Display first 20 rows to check results
df.head(20)

,Number,Requested For,Business unit,Assigned to,Assignment group,Active,State,Priority,Category,Channel,...,Created,Updated,Resolved,Closed,Reopen count,Reassignment count,Change,Value,Start,End
0,INC684078,QH06HW,Department DVL,NaN,Team OLW,True,New,4 - Low,NaN,Self-service,...,2025-11-05 22:05:26,2025-11-05 22:05:26,NaT,NaT,0,0,Assignment Group,Team OLW,2025-11-05 22:05:26,NaT
1,INC824935,HUHUCS,Department DKV,NaN,Team OLW,True,New,4 - Low,NaN,Self-service,...,2025-10-30 02:01:23,2025-10-30 02:01:23,NaT,NaT,0,0,Assignment Group,Team OLW,2025-10-30 02:01:23,NaT
2,INC700227,03R5DI,Department WHS,NaN,Team OLW,True,New,4 - Low,NaN,Phone,...,2025-10-30 03:02:24,2025-10-30 03:02:24,NaT,NaT,0,0,Assignment Group,Team OLW,2025-10-30 03:02:24,NaT
3,INC684248,4W53ZC,Department DKV,NaN,Team SWO,True,New,4 - Low,Software,Phone,...,2025-10-27 12:10:52,2025-10-27 12:10:52,NaT,NaT,0,0,Assignment Group,Team SWO,2025-10-27 12:10:52,NaT
4,INC333986,BJ66DH,Department LUF,NaN,Team UGY,True,New,4 - Low,Inquiry,Email,...,2025-11-08 09:31:27,2025-11-08 09:31:27,NaT,NaT,0,0,Assignment Group,Team UGY,2025-11-08 09:31:27,NaT
5,INC387478,BJ66DH,Department LUF,NaN,Team UGY,True,New,4 - Low,Inquiry,Email,...,2025-10-29 15:48:45,2025-10-29 15:48:45,NaT,NaT,0,0,Assignment Group,Team UGY,2025-10-29 15:48:45,NaT
6,INC775871,XVAKQM,Department KKT,NaN,Team OLW,True,New,4 - Low,NaN,Self-service,...,2025-11-04 08:22:04,2025-11-04 08:22:04,NaT,NaT,0,0,Assignment Group,Team OLW,2025-11-04 08:22:04,NaT
7,INC256704,ZV2AAB,Department FPQ,NaN,Team OLW,True,New,4 - Low,NaN,Self-service,...,2025-10-28 03:36:46,2025-10-28 03:36:46,NaT,NaT,0,0,Assignment Group,Team OLW,2025-10-28 03:36:46,NaT
8,INC257033,0QSVF3,Department JVF,NaN,Team GRK,True,New,4 - Low,Network Infrastructure,Phone,...,2025-10-25 14:15:32,2025-10-25 14:15:32,NaT,NaT,0,0,Assignment Group,Team GRK,2025-10-25 14:15:32,NaT
9,INC829164,T18SW8,Department REW,NaN,Team PUV,True,Resolved,4 - Low,Software,Chat,...,2025-10-23 00:48:39,2025-10-23 01:49:43,2025-10-23 01:49:43,NaT,0,0,Start,NaN,2025-10-23 00:48:39,2025-10-23 01:49:43


In [10]:
# Display last 20 rows to check results
df.tail(20)

,Number,Requested For,Business unit,Assigned to,Assignment group,Active,State,Priority,Category,Channel,...,Created,Updated,Resolved,Closed,Reopen count,Reassignment count,Change,Value,Start,End
23790,INC375155,FXXNE0,Department JVF,A0SGFT,Team NFL,False,Closed,4 - Low,Network Device,Phone,...,2025-09-27 08:49:26,2025-10-27 13:42:49,2025-10-22 13:06:29,2025-10-27 13:42:49,0,0,Assignment Group,Team NFL,2025-09-27 08:49:26,2025-10-27 13:42:49
23791,INC348254,VSJFJW,Department VYP,M8ATYX,Team EKX,False,Closed,4 - Low,Computer,Self-service,...,2025-09-21 12:37:12,2025-10-14 15:40:55,2025-10-09 14:53:50,2025-10-14 15:40:55,0,1,Start,NaN,2025-09-21 12:37:12,2025-10-09 14:53:50
23792,INC348254,VSJFJW,Department VYP,M8ATYX,Team EKX,False,Closed,4 - Low,Computer,Self-service,...,2025-09-19 15:28:29,2025-10-12 18:32:12,2025-10-07 17:45:07,2025-10-12 18:32:12,0,1,Assigned to,M8ATYX,2025-09-22 18:10:31,2025-10-12 18:32:12
23793,INC348254,VSJFJW,Department VYP,M8ATYX,Team EKX,False,Closed,4 - Low,Computer,Self-service,...,2025-09-22 22:54:38,2025-10-16 01:58:21,2025-10-11 01:11:16,2025-10-16 01:58:21,0,1,Assignment Group,Team NEG,2025-09-22 22:54:38,2025-09-23 00:47:29
23794,INC348254,VSJFJW,Department VYP,M8ATYX,Team EKX,False,Closed,4 - Low,Computer,Self-service,...,2025-09-13 20:34:01,2025-10-06 23:37:44,2025-10-01 22:50:39,2025-10-06 23:37:44,0,1,Assignment Group,Team EKX,2025-09-13 22:26:52,2025-10-06 23:37:44
23795,INC455972,68GYOR,Department HOE,PH7H6L,Team GKX,False,Closed,4 - Low,Inquiry,Email,...,2025-09-24 19:03:51,2025-10-04 18:10:45,2025-09-29 17:54:18,2025-10-04 18:10:45,0,0,Start,NaN,2025-09-24 19:03:51,2025-09-29 17:54:18
23796,INC455972,68GYOR,Department HOE,PH7H6L,Team GKX,False,Closed,4 - Low,Inquiry,Email,...,2025-09-21 07:19:44,2025-10-01 06:26:38,2025-09-26 06:10:11,2025-10-01 06:26:38,0,0,Assigned to,PH7H6L,2025-09-24 07:07:01,2025-10-01 06:26:38
23797,INC455972,68GYOR,Department HOE,PH7H6L,Team GKX,False,Closed,4 - Low,Inquiry,Email,...,2025-09-25 02:37:15,2025-10-05 01:44:09,2025-09-30 01:27:42,2025-10-05 01:44:09,0,0,Assignment Group,Team GKX,2025-09-25 02:37:15,2025-10-05 01:44:09
23798,INC658871,6LIB3Q,Department LUF,DNOCRH,Team UGY,False,Closed,4 - Low,Inquiry,Email,...,2025-09-20 22:44:18,2025-10-21 04:51:15,2025-10-16 04:17:24,2025-10-21 04:51:15,0,0,Start,NaN,2025-09-20 22:44:18,2025-10-16 04:17:24
23799,INC658871,6LIB3Q,Department LUF,DNOCRH,Team UGY,False,Closed,4 - Low,Inquiry,Email,...,2025-09-27 06:15:23,2025-10-27 12:22:20,2025-10-22 11:48:29,2025-10-27 12:22:20,0,0,Assigned to,DNOCRH,2025-09-27 08:36:34,2025-10-27 12:22:20
